In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from keras import optimizers
import tensorflow as tf

# 데이터 읽기

In [ ]:
origin_df = pd.read_csv("stock_price_API/newquant_samsung.csv",index_col='Date',parse_dates=True)
origin_df = origin_df[['Close', 'kosADX14', 'kosADX30', 'kosADX60', 'kosADX90', 'CCI30', 'CCI60', 'MTM1', 'MTM3', 'ROC60', 'ROC90', 'WPR60', 'WPR90', 'ADX14', 'ADX30', 'ADX60', 'ADX90', 'ADXR14', 'ADXR30', 'ADXR60', 'ADXR90']]

FileNotFoundError: ignored

In [ ]:
origin_df.head()

In [ ]:
origin_df.tail()

In [ ]:
origin_df['next_price'] = origin_df['Close'].shift(-90)

In [ ]:
origin_df.shape

In [ ]:
origin_df.info()

* null값 처리

In [ ]:
origin_df.isnull().sum()

In [ ]:
df = origin_df.dropna()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.loc['2011':'2020']

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

* 라벨링

In [ ]:
diffs = df['next_price']-df['Close']
df['label'] = np.where(diffs>0,1,0)

In [ ]:
df['label'].value_counts().plot.bar()

# 간단한 탐색

In [ ]:
# df.describe()

In [ ]:
# plt.figure(figsize=(12,12))
# sns.heatmap(df.corr(),linewidth=0.1,vmax=0.5,cmap=plt.cm.gist_heat,linecolor='white',annot=True)
# plt.show()

# 데이터 전처리

In [ ]:
X = df.drop(['next_price','label'], axis=1).values
y = df['label'].values

* minmax scaler

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)

In [ ]:
np.isinf(X).sum()

# trn, tst 구분

In [ ]:
# seed 값 설정
seed = 1234
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2, random_state=seed, stratify=y)

In [ ]:
X_train.shape

# Random Forest 예측

In [ ]:
from sklearn.ensemble import RandomForestClassifier # 랜덤포레스트
# 개별 ML 모델을 위한 Classifier 생성.
rf_clf = RandomForestClassifier(criterion='entropy',n_estimators=700, random_state=1)#랜덤포레스트

In [ ]:
# 개별 모델들을 학습. 
rf_clf.fit(X_train , Y_train)

In [ ]:
from sklearn.metrics import accuracy_score
# 학습된 개별 모델들이 각자 반환하는 예측 데이터 셋을 생성하고 개별 모델의 정확도 측정. 
rf_pred = rf_clf.predict(X_test)

In [ ]:
# 피클 파일로 저장
import pickle

pickle.dump(rf_clf, open('ss_model.pkl','wb'))
